# <center>Trabajo práctico N°4</center>

Encontrar la solución discreta $u$ correspondiente al problema variacional $ \langle u',v' \rangle = \langle f,v \rangle \ \forall v \in V_0^1 (I)$

In [1]:
using LinearAlgebra
using Plots
# using SymPy

In [2]:
function get_nodes_ix(nodes, nelem)
    ix = zeros(nodes)
    for i in 1:nodes
        ix[i] = (nodes - 1) * (nelem - 1) + i
    end
    return map(y -> round(Int,y), ix)  # to avoid float indices.
end

get_nodes_ix (generic function with 1 method)

In [3]:
function simpson_rule_l(i, j, ϕ_prime)
    q = [0 1/2 1]
    weights = [1/6 4/6 1/6]
    rv = 0
    for k in 1:3
        rv = rv + ϕ_prime[i](q[k]) * ϕ_prime[j](q[k]) * weights[k]
    end
    return rv
end

simpson_rule_l (generic function with 1 method)

In [46]:
function simpson_rule_f(i, fun, h, nodes_by_element; method="linear")
    
    if method == "linear"
        rv = fun(h*(i-1) + h/2) * (h/2)
    elseif method == "quad"
        ix = Int(i%nodes_by_element) + 1
        rv = fun(h*(i-1)/2) * (h/6) * (1+3*(ix-2)*(ix-1)/2)
    else
        error("Invalid method. Either linear or quad, instead got $method")
    end
    return rv
end

simpson_rule_f (generic function with 2 methods)

In [47]:
function right_hand_side(fun, ϕ, total_nodes, h, nodes_by_element)
    rv = Array{Float64}(undef, total_nodes)
    for j in 1:total_nodes
        rv[j] = simpson_rule_f(j, fun, h, nodes_by_element)
    end
    return rv
end

right_hand_side (generic function with 2 methods)

In [63]:
function finite_elements(nelem, fun, ϕ, ϕ_prime; method="linear")
    @assert (method in ["linear", "quad"])
    h = 1/nelem
    nodes_by_element = size(ϕ)[2]
    
    # local matrix
    local_matrix = zeros(nodes_by_element, nodes_by_element)
    for i in 1:nodes_by_element, j in 1:nodes_by_element
        local_matrix[i, j] = simpson_rule_l(i, j, ϕ_prime)
    end

    # global matrix
    F = zeros(nelem + 1)
    total_nodes = nelem*(nodes_by_element-1) + 1
    global_matrix = zeros(nelem + 1, nelem + 1)
    for i in 1:nelem
        nodes_ix = get_nodes_ix(nodes_by_element, i)
        i1, i2 = nodes_ix[1], nodes_ix[2]
        global_matrix[i1:i2, i1:i2] = global_matrix[i1:i2, i1:i2] + local_matrix
        
        if method == "linear"
            aux = right_hand_side(fun, ϕ, total_nodes, h)[i]
        elseif method == "quad"
            aux = right_hand_side(fun, ϕ, total_nodes, h, nodes_by_element)[i1:i2]
        else
            error("Invalid method. Either linear or quad, instead got $method")
        end

        F[i1:i2] = F[i1:i2] .+ aux
    end
    
    F[1], F[end] = 0, 0
    global_matrix = global_matrix .* h
    
    # border conditions
    global_matrix[2:end, 1] .= 0
    global_matrix[1, 2:end] .= 0
    global_matrix[1, 1] = 1
    
    global_matrix[1:end-1, end] .= 0
    global_matrix[end, 1:end-1] .= 0
    global_matrix[end, end] = 1

    return (global_matrix \ F) .* h
end

finite_elements (generic function with 1 method)

In [64]:
ϕ_1 = x -> x
ϕ_2 = x -> 1 - x
ϕ = [ϕ_1 ϕ_2]

ϕ_1_prime = x -> -1
ϕ_2_prime = x -> 1
ϕ_prime = [ϕ_1_prime ϕ_2_prime]

1×2 Matrix{Function}:
 #255  #257

In [65]:
g = x -> (1/2)*x*(1-x)
g2 = x -> (1/6)*(x - x^3)
g3 = x -> (exp(1)-1)*x-exp(x)+1

h2 = 1/200
p = zeros(201)
for i in 1:201
    p[i]=(i-1)*h2
end

UNAA_COSA = 16

h3 = 1/UNAA_COSA
v = zeros(UNAA_COSA+1)
for i in 1:UNAA_COSA+1
    v[i]=(i-1)*h3
end

u = finite_elements(UNAA_COSA, x->1, ϕ, ϕ_prime) ./ UNAA_COSA

plotly()
plot(p, g.(p))
plot!(v, u, seriestype = :scatter)

# quad

In [66]:
quad_ϕ_1 = x ->2*(x-(1/2))*(x-1)
quad_ϕ_2 = x ->-4*x*(x-1)
quad_ϕ_3 = x ->2*x*(x-(1/2))
quad_ϕ = [quad_ϕ_1 quad_ϕ_2 quad_ϕ_3]

quad_ϕ_prime_1 = x -> 2*(x-1)+ 2*(x-(1/2))
quad_ϕ_prime_2 = x -> -4*x-4*(x-1)
quad_ϕ_prime_3 = x -> 2*x + 2*(x-(1/2))

quad_ϕ_prime = [quad_ϕ_prime_1 quad_ϕ_prime_2 quad_ϕ_prime_3]

1×3 Matrix{Function}:
 #273  #275  #277

In [67]:
_u = finite_elements(UNAA_COSA, x->1, quad_ϕ, quad_ϕ_prime; method="quad")

LoadError: DimensionMismatch("dimensions must match: a has dims (Base.OneTo(2), Base.OneTo(2)), b has dims (Base.OneTo(3), Base.OneTo(3)), mismatch at 1")